<a href="https://colab.research.google.com/gist/optilearning/ef29bb17bcfd52e3dfc4e7d2bbde6a0b/sql-basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL

SQL, or Structured Query Language, is a powerful language used to query and modify relational databases (often pronounced "sequel"). It's not a general-purpose programming language that lets you create whole programs. To enable database interaction for any application, SQL queries may be called from programming languages. The fundamental statements we will be studying today are supported by all of the different SQL variations.

Initially, we will set up SQL on Python, which is the recommended way to use SQL for this program. The existing library, sqlite3 allows us to integrate SQL with Python.

In [ ]:
import sqlite3
import pandas as pd

To run the following commands, please download `survey.db` database. This survey contains multiple tables with scientist in Antarctica measuring different chemical values.

In [ ]:
# connecting to a dataset (this could be a .sqlite or .db file)
con = sqlite3.connect("survey.db")

For now, let’s write an SQL query that displays scientists’ names. We do this using the SQL command SELECT, giving it the names of the columns we want and the table we want them from. In this case, we want to display all tables.

## SELECT, FROM, WHERE

In [ ]:
# SQL stores data in a tabular format

# taking a look at tables will be done using pandas.read_sql option, and SELECT.
tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", con)

In [ ]:
# show tables
tables

,type,name,tbl_name,rootpage,sql
0,table,Person,Person,2,"CREATE TABLE Person (id text, personal text, f..."
1,table,Site,Site,3,"CREATE TABLE Site (name text, lat real, long r..."
2,table,Survey,Survey,5,"CREATE TABLE Survey (taken integer, person tex..."
3,table,Visited,Visited,4,"CREATE TABLE Visited (id integer, site text, d..."


In [ ]:
# filter for person column
person = pd.read_sql("""SELECT *
                        FROM Person;""", con)
person

,id,personal,family
0,dyer,William,Dyer
1,pb,Frank,Pabodie
2,lake,Anderson,Lake
3,roe,Valentina,Roerich
4,danforth,Frank,Danforth


In [ ]:
# and if we want to further filter for personal name, we'd indicate it in the command
personal = pd.read_sql("""SELECT personal
                        FROM Person;""", con)
personal

,personal
0,William
1,Frank
2,Anderson
3,Valentina
4,Frank


SQL is case-insensitive, unlike Python. Whether you capitalize `SELECT` or not, it would still work. However, the standardized conventions include capitalizing SQL commands, Title-Capitalizing tables, and lowercase everything else.

In [ ]:
# let's examine the quantities measured in the Survey table
quantities = pd.read_sql("""SELECT quant
                        FROM Survey;""", con)
quantities

,quant
0,rad
1,sal
2,rad
3,sal
4,rad
5,sal
6,temp
7,rad
8,sal
9,temp


## DISTINCT

As we see, the quantities measured are redundant. If we would like to see the unique quantities measured, we use `DISTINCT`.


In [ ]:
unique_quantities = pd.read_sql("""SELECT DISTINCT quant
                        FROM Survey""", con)
unique_quantities

,quant
0,rad
1,sal
2,temp


This provides us a much clearer view of quantities measured.

If we want to determine which visit (stored in the `taken` column) have which quant measurement, we can use the `DISTINCT` keyword on multiple columns. If we select more than one column, distinct sets of values are returned (in this case pairs, because we are selecting two columns):

In [ ]:
unique_quantities_visit = pd.read_sql("""SELECT DISTINCT taken, quant
                        FROM Survey;""", con)
unique_quantities_visit

,taken,quant
0,619,rad
1,619,sal
2,622,rad
3,622,sal
4,734,rad
5,734,sal
6,734,temp
7,735,rad
8,735,sal
9,735,temp


## ORDER BY, DESC, ASC

The next step is to use the `Person` database to determine who the scientists on the expedition are. Database records are not kept in any specific sequence, as was previously noted. As a result, query results aren't always sorted, and even when they are, we frequently wish to order them differently, such by their identifier rather than by their name. By using an `ORDER BY` clause in our SQL query, we may do this:

In [ ]:
ordered_scientists = pd.read_sql("""SELECT * FROM Person 
                                    ORDER BY id""", con)
ordered_scientists 

,id,personal,family
0,danforth,Frank,Danforth
1,dyer,William,Dyer
2,lake,Anderson,Lake
3,pb,Frank,Pabodie
4,roe,Valentina,Roerich


By default, when we use `ORDER BY`, results are sorted in ascending order of the column we specify (i.e., from least to greatest).

We can sort in the opposite order using `DESC` (for “descending”):

In [ ]:
ordered_scientists_desc = pd.read_sql("""SELECT * FROM Person 
                                    ORDER BY id DESC""", con)
ordered_scientists_desc

,id,personal,family
0,roe,Valentina,Roerich
1,pb,Frank,Pabodie
2,lake,Anderson,Lake
3,dyer,William,Dyer
4,danforth,Frank,Danforth


We may once more look at the Survey table to see which scientist measured what throughout each visit. We can also perform a multi-field sort. As an illustration, the following query arranges the results first by taken, then by person within each group of equal taken values:

In [ ]:
quant_scientist_visit = pd.read_sql("""SELECT taken, person, quant
                                    FROM Survey 
                                    ORDER BY taken ASC, person DESC""", con)
quant_scientist_visit

,taken,person,quant
0,619,dyer,rad
1,619,dyer,sal
2,622,dyer,rad
3,622,dyer,sal
4,734,pb,rad
5,734,pb,temp
6,734,lake,sal
7,735,pb,rad
8,735,None,sal
9,735,None,temp


## Boolean Values, AND, OR, LIKE


The capacity to filter data, or choose just records that fit specific criteria, is one of a database's most potent functions. Consider the scenario where we wish to know when a specific site was visited. By adding a `WHERE` clause to our query, we may choose these records from the Visited table:

In [ ]:
pd.read_sql("""SELECT * FROM Visited
               WHERE site='DR-1' """, con)

,id,site,dated
0,619,DR-1,1927-02-08
1,622,DR-1,1927-02-10
2,844,DR-1,1932-03-22


In [ ]:
# select ID feom specific site
pd.read_sql("""SELECT id FROM Visited
               WHERE site='DR-1' """, con)

,id
0,619
1,622
2,844


In [ ]:
# we can make use of booleans to filter as well (here we filter for site and date)
pd.read_sql("""SELECT * FROM Visited
               WHERE site='DR-1' AND dated < '1930-01-01' """, con)

,id,site,dated
0,619,DR-1,1927-02-08
1,622,DR-1,1927-02-10


In [ ]:
# we can also use OR to output mutually inclusive statements
pd.read_sql("""SELECT * FROM Survey
               WHERE person='lake' OR person='roe' """, con)

,taken,person,quant,reading
0,734,lake,sal,0.05
1,751,lake,sal,0.10
2,752,lake,rad,2.19
3,752,lake,sal,0.09
4,752,lake,temp,-16.00
5,752,roe,sal,41.60
6,837,lake,rad,1.46
7,837,lake,sal,0.21
8,837,roe,sal,22.50
9,844,roe,rad,11.25


In [ ]:
# we can also use IN for similar purposes
pd.read_sql("""SELECT * FROM Survey
               WHERE person in ('lake', 'roe') """, con)

,taken,person,quant,reading
0,734,lake,sal,0.05
1,751,lake,sal,0.10
2,752,lake,rad,2.19
3,752,lake,sal,0.09
4,752,lake,temp,-16.00
5,752,roe,sal,41.60
6,837,lake,rad,1.46
7,837,lake,sal,0.21
8,837,roe,sal,22.50
9,844,roe,rad,11.25


If you'd like to combine booleans, make sure to use parenthesis for desired results.

On the other hand, we use `LIKE` for partial matching. A great example of that is when you'd like to find all sites that start with 'DR'. Let's see how we can do that.

In [ ]:
pd.read_sql("""SELECT * FROM Visited
               WHERE site LIKE 'DR%' """, con)

,id,site,dated
0,619,DR-1,1927-02-08
1,622,DR-1,1927-02-10
2,734,DR-3,1930-01-07
3,735,DR-3,1930-01-12
4,751,DR-3,1930-02-26
5,752,DR-3,None
6,844,DR-1,1932-03-22


In [ ]:
# creating/calculating new values can be done by simply using numbers and operations 

pd.read_sql("""SELECT 1.05 * reading FROM Survey 
              WHERE quant = 'rad' """, con)

,1.05 * reading
0,10.3110
1,8.1900
2,8.8305
3,7.5810
4,4.5675
5,2.2995
6,1.5330
7,11.8125


In [ ]:
# we can convert temperature readings from Fahrenheit to Celsius and round to two decimal places:

pd.read_sql("""SELECT taken, round(5 * (reading - 32) / 9, 2) 
           FROM Survey WHERE quant = 'temp' """, con)

,taken,"round(5 * (reading - 32) / 9, 2)"
0,734,-29.72
1,735,-32.22
2,751,-28.06
3,752,-26.67


However, as this example demonstrates, the string representing our new field (produced by the equation) might get pretty confusing. Any field, whether computed or an already-existing column in our database, can be renamed using SQL for greater clarity. For instance, the prior inquiry may be expressed as:

In [ ]:
pd.read_sql("""SELECT taken, round(5 * (reading - 32) / 9, 2) 
              as Celsius FROM Survey WHERE quant = 'temp'""", con)

,taken,Celsius
0,734,-29.72
1,735,-32.22
2,751,-28.06
3,752,-26.67


Values from many fields can also be combined, for instance by using the string concatenation operator ||:

In [ ]:
pd.read_sql("""SELECT personal || ' ' || family FROM Person""", con)

,personal || ' ' || family
0,William Dyer
1,Frank Pabodie
2,Anderson Lake
3,Valentina Roerich
4,Frank Danforth


## NULL Values

Real-world data always has missing values; it is never fully comprehensive. Databases use a unique value called null to indicate these values. Null is a unique value that denotes "nothing here," unlike zero, False, or the empty string. Null handling calls for a few unique techniques as well as some careful consideration.

SQLite's output does not by default include NULL values. The value you give for NULLs is shown by SQLite when you execute the.nullvalue command. To make the NULLs clearer to notice, we'll use the value -null:

In [ ]:
# note the None (or null) value at row 5.
pd.read_sql("""SELECT * FROM Visited""", con)

,id,site,dated
0,619,DR-1,1927-02-08
1,622,DR-1,1927-02-10
2,734,DR-3,1930-01-07
3,735,DR-3,1930-01-12
4,751,DR-3,1930-02-26
5,752,DR-3,None
6,837,MSK-4,1932-01-14
7,844,DR-1,1932-03-22


In [ ]:
# to filter for Null, use WHERE and IS
pd.read_sql("""SELECT * FROM Visited WHERE dated IS NULL""", con)

,id,site,dated
0,752,DR-3,None


In [ ]:
# to filter for all non-Null values, we use the came code with NOT
pd.read_sql("""SELECT * FROM Visited WHERE dated IS NOT NULL""", con)

,id,site,dated
0,619,DR-1,1927-02-08
1,622,DR-1,1927-02-10
2,734,DR-3,1930-01-07
3,735,DR-3,1930-01-12
4,751,DR-3,1930-02-26
5,837,MSK-4,1932-01-14
6,844,DR-1,1932-03-22


Wherever they exist, null values may be troublesome. Consider the scenario where we wish to locate all the salinity readings that Lake didn't take. It makes sense to format the inquiry as follows:

In [ ]:
pd.read_sql("""SELECT * FROM Survey WHERE quant = 'sal' AND person != 'lake' """, con)

,taken,person,quant,reading
0,619,dyer,sal,0.13
1,622,dyer,sal,0.09
2,752,roe,sal,41.60
3,837,roe,sal,22.50


However, the data where we don't know who took the measurement are excluded by this query filter. The record isn't maintained in our results since, once again, when person is null, the comparison yields null. We must provide an explicit check whether we intend to maintain these records:

In [ ]:
pd.read_sql("""SELECT * FROM Survey WHERE quant = 'sal' AND (person != 'lake' OR person IS NULL) """, con)

,taken,person,quant,reading
0,619,dyer,sal,0.13
1,622,dyer,sal,0.09
2,735,None,sal,0.06
3,752,roe,sal,41.60
4,837,roe,sal,22.50


Similarly, all other arithmetic and boolean methods (such as `min`, `max`, `sum`, etc.), ignore Null values. This is why we need to make sure to decide what to do with these values beforehand.

## MIN, MAX, AVG, COUNT, SUM, GROUP BY 

We will calculate sums, averages, ranges and other summary values.

In [ ]:
pd.read_sql("""SELECT dated FROM Visited""", con)

,dated
0,1927-02-08
1,1927-02-10
2,1930-01-07
3,1930-01-12
4,1930-02-26
5,None
6,1932-01-14
7,1932-03-22


To summarize date data, we will use `min` and max.

In [ ]:
pd.read_sql("""SELECT min(dated) FROM Visited""", con)

,min(dated)
0,1927-02-08


In [ ]:
pd.read_sql("""SELECT max(dated) FROM Visited""", con)

,max(dated)
0,1932-03-22


To find the average, count and sum of measurements readings, we will use the following functions:

In [ ]:
# avg
pd.read_sql("""SELECT avg(reading) FROM Survey WHERE quant = 'sal'""", con)

,avg(reading)
0,7.203333


In [ ]:
# count
pd.read_sql("""SELECT count(reading) FROM Survey WHERE quant = 'sal'""", con)

,count(reading)
0,9


In [ ]:
 # sum
pd.read_sql("""SELECT sum(reading) FROM Survey WHERE quant = 'sal'""", con)

,sum(reading)
0,64.83


Multiple aggregations at once:

In [ ]:
pd.read_sql("""SELECT min(reading), max(reading) FROM Survey 
               WHERE quant = 'sal' AND reading <= 1.0""", con)

,min(reading),max(reading)
0,0.05,0.21


Mixing raw and aggregated data can become tricky. SQL will choose one value from the raw data to aggregate from, which more often than not is not useful. Let's see an example of that.

In [ ]:
pd.read_sql("""SELECT person, count(reading), round(avg(reading), 2)
FROM  Survey
WHERE quant = 'rad'""", con)

,person,count(reading),"round(avg(reading), 2)"
0,roe,8,6.56


Instead of aggregating independently for each scientist, the database manager chooses a single name at random. We could create five queries of the following kind because there are only five scientists:

In [ ]:
pd.read_sql("""SELECT person, count(reading), round(avg(reading), 2)
FROM  Survey
WHERE quant = 'rad'
AND person = 'dyer' """, con)

,person,count(reading),"round(avg(reading), 2)"
0,dyer,2,8.81


At scale, this solution proves to be tedious. Imagine having 100 scientists, and having to write 100 separate queries. Luckily, there's no need for that, as SQL has an in-built command, namely `GROUP BY`.

In [ ]:
pd.read_sql("""SELECT person, count(reading), round(avg(reading), 2)
FROM  Survey
WHERE quant = 'rad'
GROUP BY person """, con)

,person,count(reading),"round(avg(reading), 2)"
0,dyer,2,8.81
1,lake,2,1.82
2,pb,3,6.66
3,roe,1,11.25


We can also group by multiple criteria. For example:

In [ ]:
pd.read_sql("""SELECT person, count(reading), round(avg(reading), 2)
FROM  Survey
GROUP BY person, quant """, con)

,person,count(reading),"round(avg(reading), 2)"
0,None,1,0.06
1,None,1,-26.00
2,dyer,2,8.81
3,dyer,2,0.11
4,lake,2,1.82
5,lake,4,0.11
6,lake,1,-16.00
7,pb,3,6.66
8,pb,2,-20.00
9,roe,1,11.25


To remove all measurements where we do not know which scientist took them, we add the null-value filtering inside the command.

In [ ]:
pd.read_sql("""SELECT   person, quant, count(reading), round(avg(reading), 2)
FROM     Survey
WHERE    person IS NOT NULL
GROUP BY person, quant
ORDER BY person, quant""", con)

,person,quant,count(reading),"round(avg(reading), 2)"
0,dyer,rad,2,8.81
1,dyer,sal,2,0.11
2,lake,rad,2,1.82
3,lake,sal,4,0.11
4,lake,temp,1,-16.00
5,pb,rad,3,6.66
6,pb,temp,2,-20.00
7,roe,rad,1,11.25
8,roe,sal,2,32.05


When we take a closer look, we find that:

* Survey table records were picked out where the person field wasn't null;

* records were divided into subsets, each of which had the identical person and quant values;

* subsets were sorted by person first, then by quant within each subgroup; and

* a person and a quant value were selected from each subgroup (it doesn't matter which ones, as they are all identical), counted the number of entries in each subset, computed the average reading in each, and then did the same for the whole dataset.

## JOIN

SQL uses tables for data. Sometimes, we need to combine information from different tables. Let's see how that works:

In [ ]:
pd.read_sql("""SELECT * FROM Site JOIN Visited""", con)

,name,lat,long,id,site,dated
0,DR-1,-49.85,-128.57,619,DR-1,1927-02-08
1,DR-1,-49.85,-128.57,622,DR-1,1927-02-10
2,DR-1,-49.85,-128.57,734,DR-3,1930-01-07
3,DR-1,-49.85,-128.57,735,DR-3,1930-01-12
4,DR-1,-49.85,-128.57,751,DR-3,1930-02-26
5,DR-1,-49.85,-128.57,752,DR-3,None
6,DR-1,-49.85,-128.57,837,MSK-4,1932-01-14
7,DR-1,-49.85,-128.57,844,DR-1,1932-03-22
8,DR-3,-47.15,-126.72,619,DR-1,1927-02-08
9,DR-3,-47.15,-126.72,622,DR-1,1927-02-10


`JOIN` combines each record from one table with each record from the other to produce the cross product of the two tables, which provides all conceivable combinations. Three records in `Site` and eight in `Visited` make up the output of the join, which contains 24 records (3 * 8 = 24). Because there are three entries in each table, the output includes six fields (3 + 3 = 6).

`JOIN` hasn't determined if the records it is joining are related to one another. Until we show it how, it has no means of knowing whether they do or not. In order to do so, we add a condition stating that we're only interested in combinations that have the same site name, hence we must make use of a filter.

In [ ]:
pd.read_sql("""SELECT Site.lat, Site.long, Visited.dated
              FROM Site
              JOIN Visited ON Site.name = Visited.site""", con)

,lat,long,dated
0,-49.85,-128.57,1927-02-08
1,-49.85,-128.57,1927-02-10
2,-49.85,-128.57,1932-03-22
3,-47.15,-126.72,None
4,-47.15,-126.72,1930-01-07
5,-47.15,-126.72,1930-01-12
6,-47.15,-126.72,1930-02-26
7,-48.87,-123.40,1932-01-14


Joining several tables must be better if joining two tables is good. In fact, by simply adding additional `JOIN` clauses to our query and more `ON` checks to exclude record combinations that are illogical, we can join any number of tables.

In [ ]:
pd.read_sql("""SELECT Site.lat, Site.long, Visited.dated, Survey.quant, Survey.reading
              FROM Site
              JOIN Visited
              JOIN Survey ON Site.name = Visited.site
              AND Visited.id = Survey.taken
              AND Visited.dated IS NOT NULL""", con)

,lat,long,dated,quant,reading
0,-49.85,-128.57,1927-02-08,rad,9.82
1,-49.85,-128.57,1927-02-08,sal,0.13
2,-49.85,-128.57,1927-02-10,rad,7.80
3,-49.85,-128.57,1927-02-10,sal,0.09
4,-47.15,-126.72,1930-01-07,rad,8.41
5,-47.15,-126.72,1930-01-07,sal,0.05
6,-47.15,-126.72,1930-01-07,temp,-21.50
7,-47.15,-126.72,1930-01-12,rad,7.22
8,-47.15,-126.72,1930-01-12,sal,0.06
9,-47.15,-126.72,1930-01-12,temp,-26.00


Due to the presence of primary keys and foreign keys in the Site, Visited, and Survey tables, we can determine which entries belong to one another. A primary key is a value—or a set of values—that enables each entry in a database to be identified only once. A value (or combination of values) from one table that uniquely identifies a record in another table is known as a foreign key. Another way of putting this is that a foreign key is the primary key of one table that occurs in some other table. In our database, Person.id serves as the Person table's primary key, while Survey.person serves as a foreign key connecting entries in the Survey table to entries in Person.

## Formatting databases


We can now understand why and how to utilize the relational model because we have seen how joins operate. Every value must be atomic, or free of components that we might desire to manipulate individually, according to the first criterion. To avoid having to utilize substring operations to retrieve the name's components, we keep personal and family names separately rather than having them all in one column. Furthermore, dividing on spaces is unreliable—just consider names like "Maryn Jr. King" or "Leon Moore Jackson"—so we record the two halves of the name separately.

The requirement that each record have a distinct primary key is the second criterion. The triple (taken, person, quant) from the Survey database uniquely identifies each measurement. This can be a serial number with no inherent significance, one of the values in the record (like the id field in the Person table), or even a mix of values.

There should be no duplicate information, which is the third criterion. We may, for instance, eliminate the Site table and replace the Visited table from above, as they present redundant information.